In [11]:
from feast import FeatureStore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from joblib import dump
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
# Getting our FeatureStore
store = FeatureStore(repo_path=".")

In [13]:
# Retrieving the saved dataset and converting it to a DataFrame
training_df = store.get_saved_dataset(name="churn_dataset").to_df()
training_df

C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\feast\feature_store.py:1178: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,Geography_Spain,event_timestamp,USER_ID,HasCrCard,Balance,Age,Exited,Gender,Tenure,EstimatedSalary,CreditScore,NumOfProducts,Geography_France,IsActiveMember,Geography_Germany
0,False,2023-01-01 00:00:00+00:00,9417,1,131753.41,37,0,1,6,86894.67,648,1,False,0,True
1,False,2023-01-01 00:00:00+00:00,6186,1,147137.74,36,0,1,6,33687.90,699,1,False,1,True
2,False,2023-01-01 00:00:00+00:00,792,1,0.00,49,0,1,3,142917.54,611,2,True,1,False
3,False,2023-01-01 00:00:00+00:00,9133,1,0.00,33,0,0,5,122949.71,635,2,True,0,False
4,False,2023-01-01 00:00:00+00:00,9145,1,105301.50,43,0,1,10,78941.59,818,1,False,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,2023-12-31 00:00:00+00:00,9329,1,128369.75,35,0,0,7,198256.15,540,2,False,0,True
9996,True,2023-12-31 00:00:00+00:00,7324,1,0.00,24,0,1,8,4070.28,742,2,False,0,False
9997,False,2023-12-31 00:00:00+00:00,7966,1,124620.01,51,1,0,4,92243.94,625,2,True,0,False
9998,True,2023-12-31 00:00:00+00:00,5255,1,204223.03,20,0,1,8,128268.39,463,1,False,0,False


In [14]:
# Separating the features and labels
labels = training_df['Exited']
features = training_df.drop(
    labels=['Exited', 'event_timestamp', "USER_ID"],
    axis=1)

In [15]:
# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    stratify=labels)

In [16]:
# Supponiamo che queste siano le colonne usate durante l'addestramento
columns_order = X_train.columns
columns_order

Index(['Geography_Spain', 'HasCrCard', 'Balance', 'Age', 'Gender', 'Tenure',
       'EstimatedSalary', 'CreditScore', 'NumOfProducts', 'Geography_France',
       'IsActiveMember', 'Geography_Germany'],
      dtype='object')

In [17]:
# Riorganizza le colonne di X_test in base all'ordine delle colonne utilizzate durante l'addestramento
X_test = X_test[columns_order]
X_train = X_train[columns_order]

In [18]:
# Inizializza il modello
model = RandomForestClassifier(n_estimators=100, random_state=42)
# Addestra il modello
model.fit(X=X_train, y=y_train)

RandomForestClassifier(random_state=42)

In [19]:
# Salva l'ordine delle colonne in un file di testo
with open("columns_order.txt", "w") as file:
    for column in columns_order:
        file.write(f"{column}\n")

dump(model, 'random_forest_model.joblib')

['random_forest_model.joblib']

In [20]:
dump(X_train.columns, 'columns_order.joblib')

['columns_order.joblib']

In [21]:
X_train.columns

Index(['Geography_Spain', 'HasCrCard', 'Balance', 'Age', 'Gender', 'Tenure',
       'EstimatedSalary', 'CreditScore', 'NumOfProducts', 'Geography_France',
       'IsActiveMember', 'Geography_Germany'],
      dtype='object')